# Local db sync
Keeps a copy of local db in sync with production. This is useful for testing write operations.

In [ ]:
collections = [
  "questions",
  "qa_data",
  "tenants",
  "multitenant_page_env_variables",
  "multitenant_page_messages",
]

Service `mongodb-community` already started, use `brew services restart mongodb-community` to restart.


In [ ]:
from ut.db import db_local, db_prd

# update the local db based on the last created at date
for collection_name in collections:
  collection = db_prd[collection_name]
  collection_local = db_local[collection_name]
  print("Updating collection:", collection_name)

  # Check if the local collection exists and has documents
  if collection_local.count_documents({}) == 0:
    print("Local collection does not exist or is empty. Creating and inserting all documents.")
    all_documents = list(collection.find({}, {"embedding_vector": 0}))
    if all_documents:
      collection_local.insert_many(all_documents)
      print("Inserted", len(all_documents), "documents into the local collection.")
  else:
    last_created_at = collection_local.find().sort("created_at", -1).limit(1)[0]["created_at"]
    print("Last created at:", last_created_at)

    new_documents = [d for d in collection.find({"created_at": {"$gt": last_created_at}}, {"embedding_vector": 0})]
    if new_documents:
      print("New documents found: ", len(new_documents))
      collection_local.insert_many(new_documents)
    else:
      print("No new documents found.")

Updating collection: questions
Last created at: 2024-12-14 01:00:52.014000
New documents found:  5620
Updating collection: conversation_summaries
Last created at: 2024-12-14 01:02:44.148000
New documents found:  3234
Updating collection: qa_data
Last created at: 2024-12-13 07:21:11.229000
New documents found:  16
Updating collection: tenants
Last created at: 2024-12-11 09:47:14.610000
No new documents found.
Updating collection: multitenant_page_env_variables
Last created at: 2024-12-11 09:47:17.656000
No new documents found.
Updating collection: multitenant_page_messages
Last created at: 2024-12-11 09:47:17.685000
No new documents found.
